In [ ]:
!(@isdefined cleanup_hook) && IJulia.push_postexecute_hook(() -> (empty!(In); empty!(Out); GC.gc(true))); cleanup_hook = true;

In [ ]:
using ProgressMeter, PyCall, PyPlot, Images, HDF5,NaNStatistics, Statistics, DSP, Lasso, JLD2
using _Data, _Math

In [ ]:
@pyimport numpy
@pyimport sklearn.decomposition as decomposition
@pyimport scipy.stats as stats
@pyimport matplotlib.colors as mpl_colors
@pyimport matplotlib.cm as cm 
@pyimport sklearn.cluster as cluster

In [ ]:
include("../../../functions/func_map.jl")
include("../../../functions/func_stat.jl")
include("../../../functions/func_data.jl")
include("../../../functions/func_plot.jl")
include("../../../functions/utils.jl")


In [ ]:
rc_params = PyDict(pyimport("matplotlib")["rcParams"]);
rc_params["font.sans-serif"] = ["Arial"];
rc_params["font.size"] = 7;
rc_params["lines.linewidth"] = 1;
rc_params["lines.markersize"] = 4;
rc_params["xtick.major.size"] = 2;
rc_params["ytick.major.size"] = 2;
rc_params["xtick.major.pad"] = 2;
rc_params["ytick.major.pad"] = 2;
# rc_params["axes.labelpad"] = 2;
rc_params["axes.spines.right"] = false
rc_params["axes.spines.top"] = false

cim(img::Matrix{UInt32}) = CairoImageSurface(img, Cairo.FORMAT_RGB24; flipxy = false) 
cim(img::Matrix{ARGB32}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{RGB24}) = cim(reinterpret(UInt32, img))
cim(img::Matrix{UInt8}) = cim(Gray.(reinterpret(N0f8, img)))
cim(img::Array{UInt8,3}) = cim(RGB24.(reinterpret(N0f8, img[:,:,1]), reinterpret(N0f8, img[:,:,2]), reinterpret(N0f8, img[:,:,3])));downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);
downsample(img,n=4) = +([img[i:n:n*div(size(img,1)-1,n), j:n:n*div(size(img,2)-1,n)] for i = 1:n, j = 1:n]...)/(n*n);

In [ ]:
data_info_dict = load("/home/chuyu/fish_place_cell_paper/figures/chuyu/figure_data_info.jld2")

data_info_all = []
data_info_tog = []
job_names = []
for key in keys(data_info_dict)
    append!(data_info_all, data_info_dict[key])
    append!(data_info_tog, [data_info_dict[key]])
    append!(job_names, [String(key)])
end

In [ ]:
analyzer = "chuyu"

# Distance to landmark

In [ ]:
peak_distance_landmark_all = []
corr_trials_place_cell_all = []
for which_data = 1:3
data_info = data_info_dict["landmark_removal"][which_data]

experiment_filename_1 = data_info[1]
server_1 = data_info[2]

experiment_filename_2 = data_info[3]
server_2 = data_info[4]

experimenter = data_info[5]

ds_save_1 = Dataset(experiment_filename_1, experimenter, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_cy_1 = Dataset(experiment_filename_1, "chuyu", gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")
ds_save_analyzer_1 = Dataset(experiment_filename_1, analyzer, gethostname() == "roli-$(server_1)" ? "/data" : "/nfs/data$(server_1)")

ds_save_2 = Dataset(experiment_filename_2, experimenter, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)") # This example dataset is on roli-9, so the path is different depending on whether you're trying to access the file from roli-9
ds_save_cy_2 = Dataset(experiment_filename_2, "chuyu", gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")
ds_save_analyzer_2 = Dataset(experiment_filename_2, analyzer, gethostname() == "roli-$(server_2)" ? "/data" : "/nfs/data$(server_2)")

    
all_files = readdir(data_path(ds_save_cy_1))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_1, candidate_filename[i])*!occursin("shuffleall", candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
println(candidate_filename[which_file])
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
info_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(info_filename, "r")
place_map_all_1 = HDF5.readmmap(file["place_map_all"])
close(file)
    
    
all_files = readdir(data_path(ds_save_cy_2))
long_name_files = all_files[findall([length(all_files[i])>6 for i in 1:length(all_files)])]
spatial_info_index = findall([long_name_files[i][1:6]=="neuron" for i in 1:length(long_name_files)])
candidate_filename = long_name_files[spatial_info_index]
which_file = [occursin(experiment_filename_2, candidate_filename[i])*occursin("A_dF", candidate_filename[i]) for i in 1:length(candidate_filename)]
println(candidate_filename[which_file])
@assert(length(candidate_filename[which_file]) == 1)
save_file_name = candidate_filename[which_file][1]
info_filename = joinpath(data_path(ds_save_cy_2), save_file_name)
file = h5open(info_filename, "r")
place_map_all_2 = HDF5.readmmap(file["place_map_all"])
close(file)


n_bins = size(place_map_all_1,1)

# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_1), "chamber_geometry_$(experiment_filename_1).h5"))
global countour_array_1 = read(orientation_correction_file,"countour_array")
close(orientation_correction_file)


# orientation-corrected background image and chamber roi image
orientation_correction_file = h5open(joinpath(data_path(ds_save_cy_2), "chamber_geometry_$(experiment_filename_2).h5"))
global countour_array_2 = read(orientation_correction_file,"countour_array")
close(orientation_correction_file)


img_bg_1 = h5open(ds_save_1, "behavior.h5"; raw = true) do file
    read(file, "img_bg")
end;

img_bg_2 = h5open(ds_save_2, "behavior.h5"; raw = true) do file
    read(file, "img_bg")
end;
    
global img_bg_end_1 = img_bg_1[:,:,end]
global img_bg_end_2 = img_bg_2[:,:,end]


global w = size(img_bg_end_1, 1)
global l = size(img_bg_end_1, 2)


save_file_name = "compare_map_results_original.h5"
compare_filename = joinpath(data_path(ds_save_cy_1), save_file_name)
file = h5open(compare_filename, "r")
confined_place_cell_index = HDF5.readmmap(file["confined_place_cell_index"])
confined_place_cell_index_1 = HDF5.readmmap(file["confined_place_cell_index_1"])
confined_place_cell_index_2 = HDF5.readmmap(file["confined_place_cell_index_2"])
peak_loc_map_1 = HDF5.readmmap(file["peak_loc_map_1"])
peak_loc_map_2 = HDF5.readmmap(file["peak_loc_map_2"])

peak_shift_mm = HDF5.readmmap(file["peak_shift_mm"])
peak_distance_center_1_mm = HDF5.readmmap(file["peak_distance_center_1_mm"])
peak_distance_edge_1_mm = HDF5.readmmap(file["peak_distance_edge_1_mm"])
corr_trials_all = HDF5.readmmap(file["corr_trials_all"])
corr_pv_all = HDF5.readmmap(file["corr_pv_all"])
corr_pv_all_normalized = HDF5.readmmap(file["corr_pv_all_normalized"])
close(file)



for_place_calculation_file = h5open(joinpath(data_path(ds_save_cy_1), "for_place_calculation_chamber_geometry_$(experiment_filename_1)_n$(n_bins).h5"))
x_bins = read(for_place_calculation_file,"x_bins")
y_bins = read(for_place_calculation_file,"y_bins")
x_digital = read(for_place_calculation_file,"x_digital")
y_digital = read(for_place_calculation_file,"y_digital")
close(for_place_calculation_file)

bin_interval = x_bins[2] - x_bins[1]


KMeans_features = cluster.KMeans(n_clusters = 6).fit(hcat(peak_loc_map_1'))
class_1 = KMeans_features.labels_;
KMeans_features = cluster.KMeans(n_clusters = 6).fit(hcat(peak_loc_map_2'))
class_2 = KMeans_features.labels_;





global mask_valid_1 = isfinite.(place_map_all_1[:,:,confined_place_cell_index[1]])
mask_valid_2 = isfinite.(place_map_all_2[:,:,confined_place_cell_index[1]])


valid_pixels = findall(mask_valid_1 .* mask_valid_2);

pv_map = fill(NaN32, size(place_map_all_1[:,:,1]))
pv_map[valid_pixels] = corr_pv_all


pv_map_normalized = fill(NaN32, size(place_map_all_1[:,:,1]))
pv_map_normalized[valid_pixels] = corr_pv_all_normalized;



figure(figsize=(8,3))
subplot(1,2,1)
scatter(peak_loc_map_1[2,whether_in(confined_place_cell_index_1, confined_place_cell_index)], peak_shift_mm, alpha=0.5)

xlabel("Distance to landmark")
ylabel("peak shift")

subplot(1,2,2)
scatter(peak_loc_map_1[2,whether_in(confined_place_cell_index_1, confined_place_cell_index)], corr_trials_all[confined_place_cell_index], alpha=0.5)

xlabel("Distance to landmark")
ylabel("PF correlation")
    pixel_size = 20*10^-3
    append!(peak_distance_landmark_all, (peak_loc_map_1[2,whether_in(confined_place_cell_index_1, confined_place_cell_index_1)] .- 14).*bin_interval.*pixel_size)
    append!(corr_trials_place_cell_all, corr_trials_all[confined_place_cell_index_1])
    println(extrema(x_digital))
    println(extrema(y_digital))
    println(extrema(peak_loc_map_1[2,:]))
end

In [ ]:
fig_path = "/home/chuyu/fish_place_cell_paper/figures/output/figure3/landmark_removal"
x = reduce(vcat, peak_distance_landmark_all)
y = reduce(vcat, corr_trials_place_cell_all)
regress_x, regress_y, y_std = py"smooth_scatter"(x,y,bins =10)

fig = figure(figsize=(1.3,1))
scatter(x, y, alpha=0.3, color="gray", s= 1)
plot(regress_x, regress_y, color="r")
fill_between(regress_x, regress_y-y_std, regress_y+y_std, alpha=0.3, color="r")
xlabel("Distance to landmark (mm)")
ylabel("PF correlation")
fig.savefig(joinpath(fig_path, "distance_landmark_PF_correlation.pdf"), bbox_inches="tight",transparent = true);